## This Job will be uploaded to lambda and run twice a day

In [120]:
import time
import pythena
import datetime
import json
import s3fs
import pandas as pd
import nltk
from nltk.corpus import stopwords

athena_client = pythena.Athena("twitterappdatalake",region="ap-southeast-2") 


In [121]:
with open("C:/Users/William Jiang/Documents/credentials.json") as f:
    d = json.load(f)
    cred_json=d["twitter_api"]
    s3_cred_json=d["s3-access"]
    
##auth using s3fs
fs = s3fs.S3FileSystem(anon=False, key=s3_cred_json['ACCESS_KEY_ID'], secret=s3_cred_json['SECRET_ACCESS_KEY'],default_fill_cache=False)

In [127]:
## Custome Aggregation function. takes in numpy array
def word_freq(arr):
    stop_words = set(stopwords.words('english'))
    stop_substring=['#','@','&',"rt","https","https.","t.co"]
    stop_words.update(["rt","https","https.","-",'.',':'])
    text = " ".join(str(tweet).lower() for tweet in arr)
    all_freq={}
    for word in text.split():
        #Set the substring flag - if any substring is found then it won't get passed
        substring_flag=0
        for substring in stop_substring:
            if substring in word:
                substring_flag=1
        if substring_flag==0:
            if word not in stop_words:
                if word in all_freq: 
                    all_freq[word] += 1
                else: 
                    all_freq[word] = 1   
    word_freq_list=sorted(all_freq.items(), key=lambda x: x[1],reverse=True)[0:30]
    return word_freq_list

In [128]:
def query_to_agg_df(string_date):
    string_query='''
    SELECT * FROM twitterdatalake t where date(date_parse(created_at,'%Y-%m-%d %H:%i:%s'))=date_parse('{0}','%Y-%m-%d')
    '''.format(string_date)
    df_oneday = athena_client.execute(string_query)
    dftest=df_oneday[0]
    df_word_freq_groupby=dftest.groupby(['partition_1','partition_0'])['text'].agg([word_freq,'count'])
    df_avg_groupby=dftest.groupby(['partition_1','partition_0'])['polarity','followers','friends','retweet_count'].mean()
    #join
    df_final_agg=df_avg_groupby.join(df_word_freq_groupby, on=['partition_1','partition_0'])
    df_final_agg=df_final_agg.rename(columns={"text": "word_freq_list","polarity": "avg_polarity","followers":"avg_followers","friends":"avg_friends","retweet_count":"avg_retweet_count"})
    
    return df_final_agg

In [129]:
for file in fs.listdir('aggregatedresultsforapp'):
    print(file)

{'Key': 'aggregatedresultsforapp/2020-07-22.csv', 'LastModified': datetime.datetime(2020, 7, 28, 10, 7, 4, tzinfo=tzutc()), 'ETag': '"b24abf1db77e66c95f8b53d9491fb7b8"', 'Size': 166774, 'StorageClass': 'STANDARD', 'type': 'file', 'size': 166774, 'name': 'aggregatedresultsforapp/2020-07-22.csv'}
{'Key': 'aggregatedresultsforapp/2020-07-23.csv', 'LastModified': datetime.datetime(2020, 7, 28, 10, 18, 39, tzinfo=tzutc()), 'ETag': '"707c34c8f79739f24c90f0258d552b69"', 'Size': 174512, 'StorageClass': 'STANDARD', 'type': 'file', 'size': 174512, 'name': 'aggregatedresultsforapp/2020-07-23.csv'}
{'Key': 'aggregatedresultsforapp/2020-07-24.csv', 'LastModified': datetime.datetime(2020, 7, 28, 10, 31, 55, tzinfo=tzutc()), 'ETag': '"beede18237886661bd10c5b11c0cf840"', 'Size': 164145, 'StorageClass': 'STANDARD', 'type': 'file', 'size': 164145, 'name': 'aggregatedresultsforapp/2020-07-24.csv'}


In [130]:
# string_today=today_date.strftime("%Y-%m-%d")
max_date=datetime.datetime.strptime("2020-07-13", "%Y-%m-%d").date()
#Find the max date in the s3 bucket
for file in fs.listdir('aggregatedresultsforapp'):
    current_string=file['Key'].split('/')[-1].split('.csv')[0]
    current_date=datetime.datetime.strptime(current_string, "%Y-%m-%d").date()
    if current_date>max_date:
        max_date=current_date

print(max_date)

2020-07-24


In [131]:
t0 = time.time()
day_before_date = datetime.date.today() - datetime.timedelta(days=2)
#If less than daybeforedate then ingest the next date else leave alone
if max_date<day_before_date:
    next_date=max_date + datetime.timedelta(days=1)
    str_next_date=next_date.strftime("%Y-%m-%d")
    df_final_agg=query_to_agg_df(str_next_date)
    file_dir='s3://aggregatedresultsforapp/'
    full_filename=file_dir+ str_next_date
    with fs.open(full_filename+ '.csv','w', encoding="utf-8", newline = '\n') as f:
        df_final_agg.to_csv(f)

t1 = time.time()
total = t1-t0
print(total)

37.00192832946777
